In [16]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pickle import dump,load

In [17]:
# pickle functions
def pickle(data, filename):
    with open(filename, 'wb') as file:
        dump(data, file)
def load_pickle(filename):
    with open(filename, 'rb') as file:
        data = load(file)
    return data

In [ ]:
# import barmm barangay data
barmm_map = gpd.read_file('raw_data/barmm_barangay.shp')[['Bgy_Code', 'Pro_Name', 'Mun_Name', 'Bgy_Name', 'geometry']]
barmm_map['geometry'] = barmm_map['geometry'].to_crs(epsg=4326)

In [2]:
# import landslide data
landslide_files = ['raw_data/landslide_lanao.shp',
             'raw_data/landslide_tawi.shp',
             'raw_data/landslide_maguindanao.shp', 
             'raw_data/landslide_basilan.shp',
             'raw_data/landslide_sulu.shp']
landslide = pd.concat([gpd.read_file(file) for file in landslide_files]).reset_index(drop=True)[['geometry']]
landslide['geometry'] = landslide['geometry'].to_crs(epsg=4326)
landslide['centroid'] = landslide['geometry'].apply(lambda x: [polygon.centroid for polygon in x])

# import flood data
flood = gpd.read_file('raw_data/flood.shp')[['FloodSusc', 'geometry']].to_crs(epsg=4326)

# import storm data
storm = gpd.read_file('raw_data/storm.shp')[['HAZ', 'geometry']]

# import faultline data
faultline = gpd.read_file('raw_data/faultline.shp')[['geometry']]

In [45]:
# add landslide feature
landslide_feature = list()
landslide_shp = [polygon for shp in landslide.geometry for polygon in shp]
for barangay in barmm_map['geometry']:
    landslide_feature.append(sum([1 if shp.intersects(barangay) else 0 
                              for shp in landslide_shp]))

# pickle result
pickle(flood_feature, 'landslide_feature.pkl')

In [51]:
# add flood feature
flood_feature = list()
flood.loc[flood['FloodSusc'] == 'High', 'FloodSusc'] = 2
flood.loc[flood['FloodSusc'] != 'High', 'FloodSusc'] = 1
flood_intensity = list(flood.FloodSusc)
flood_shp = list(storm.geometry)
for barangay in barmm_map['geometry']:
    flood_feature.append(sum([intensity if shp.intersects(barangay) else 0 
                              for intensity, shp in zip(flood_intensity, flood_shp)]))

# pickle result
pickle(flood_feature, 'flood_feature.pkl')

In [9]:
# add storm feature
storm_feature = list()
storm_intensity = list(storm.HAZ)
storm_shp = list(storm.geometry)
for barangay in barmm_map['geometry']:
    storm_feature.append(sum([intensity if shp.intersects(barangay) else 0 
                              for intensity, shp in zip(storm_intensity, storm_shp)]))
# pickle result
pickle(storm_feature, 'storm_feature.pkl')

In [23]:
# add faultline feature
faultline_feature = list()
for barangay in barmm_map['geometry']:
    faultline_feature.append(sum([1 if line.intersects(barangay) else 0 for line in faultline.geometry]))

# pickle result
pickle(faultline_feature, 'faultline_feature.pkl')

In [66]:
# append additional features
landslide_feature = load_pickle('landslide_feature.pkl')
flood_feature = load_pickle('flood_feature.pkl')
storm_feature = load_pickle('storm_feature.pkl')
faultline_feature = load_pickle('faultline_feature.pkl')

barmm_map['landslide'] = landslide_feature
barmm_map['flood'] = flood_feature
barmm_map['storm'] = storm_feature
barmm_map['faultline'] = faultline_feature

barmm_map.to_csv('hazard_complete.csv', index=False)

#### 